In [9]:

from requests import get, post
import json
import time
import pandas as pd
import os
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mp
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import Query

In [2]:
def convert_blocktime(blocktime, **kwargs):
    delta = kwargs.get('delta',6)
    if blocktime is None:
        return None
    else:
        timestr = blocktime.replace('T',' ').replace('Z','')
        return datetime.strptime(timestr[:-delta], "%Y-%m-%d %H:%M:%S")

In [3]:
#from https://dune.com/docs/api/quick-start/api-py/
BASE_URL = "https://api.dune.com/api/v1/"

def make_api_url(module, action, ID):
    """
    We shall use this function to generate a URL to call the API.
    """

    url = BASE_URL + module + "/" + str(ID) + "/" + action

    return url

def execute_query(query_id, engine="medium"):
    """
    Takes in the query ID and engine size.
    Specifying the engine size will change how quickly your query runs. 
    The default is "medium" which spends 10 credits, while "large" spends 20 credits.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """

    url = make_api_url("query", "execute", query_id)
    params = {
        "performance": engine,
    }
    response = post(url, headers=HEADER, params=params)
    execution_id = response.json()['execution_id']

    return execution_id


def execute_query_with_params(query_id, param_dict):
    """
    Takes in the query ID. And a dictionary containing parameter values.
    Calls the API to execute the query.
    Returns the execution ID of the instance which is executing the query.
    """

    url = make_api_url("query", "execute", query_id)
    response = post(url, headers=HEADER, json={"query_parameters" : param_dict})
    execution_id = response.json()['execution_id']

    return execution_id


def get_query_status(execution_id):
    """
    Takes in an execution ID.
    Fetches the status of query execution using the API
    Returns the status response object
    """

    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)

    return response


def get_query_results(execution_id):
    """
    Takes in an execution ID.
    Fetches the results returned from the query using the API
    Returns the results response object
    """

    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)

    return response


def cancel_query_execution(execution_id):
    """
    Takes in an execution ID.
    Cancels the ongoing execution of the query.
    Returns the response object.
    """

    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)

    return response

def run_query(query_id,**kwargs):
    parameters = kwargs.get('parameters',{})
    status_ping_interval = kwargs.get('status_ping_interval',30)
    query_size = kwargs.get('query_size','medium')
    if len(parameters)>0:
        execution_id = execute_query_with_params(query_id,parameters)
    else:
        execution_id = execute_query(query_id,query_size)
    
    # response = get_query_results(execution_id)
    status = get_query_status(execution_id)
    timesum=0
    while (status.json()['state'] == 'QUERY_STATE_EXECUTING') & (timesum<3600):
        print("Waiting for query to finish...")
        time.sleep(status_ping_interval)
        timesum+=status_ping_interval
        status = get_query_status(execution_id)
    
    response = get_query_results(execution_id)
    if response.json()['state'] == 'QUERY_STATE_COMPLETED':
        return pd.DataFrame(response.json()['result']['rows'])
    else:
        print(response.json()['state'])
        print('No output')
        return None

In [6]:
# queries via dune (stake season 1-6, can also manually pull data via csvs if no API key)
api_key='...' #enter your API key
HEADER = {"x-dune-api-key" : api_key}

In [30]:
hopr_all_seasons_rewards = 2037368 #HOPR rewards per account per month
hopr_node_account_mapping = 2611302 #listing and delisitng dates (month) of nodes per account

In [54]:
#union across all first six staking seasons
df_hopr_rewards=run_query(hopr_all_seasons_rewards)

Waiting for query to finish...


In [56]:
df_hopr_rewards['datestr']=[convert_blocktime(df_hopr_rewards.datemonth.iloc[i], delta=8) for i in range(0,len(df_hopr_rewards)) ]

In [57]:
df_hopr_rewards

,account,datemonth,sum_rewards,sum_txs,datestr
0,0xe000adb6f47c7461694bcb69bf51bab8087d819d,2022-12-01 00:00:00.000 UTC,976.707333,2,2022-12-01
1,0x9f1842a615a14bce5723f4c5019d6f67c889b632,2023-01-01 00:00:00.000 UTC,44.212566,1,2023-01-01
2,0xc8c3e484ddf92069eb3226df266c2c262902d835,2022-11-01 00:00:00.000 UTC,1060.426451,4,2022-11-01
3,0x9062a96b9e947b2b57283e705c97b871f5eb6561,2022-11-01 00:00:00.000 UTC,431.021007,5,2022-11-01
4,0x9d8c00b16906a78943c66f1f976a98e4e7a36909,2022-11-01 00:00:00.000 UTC,1942.243035,1,2022-11-01
...,...,...,...,...,...
9012,0x7e50347f8c294a126978159eca243a741c9546ea,2021-12-01 00:00:00.000 UTC,217.170688,1,2021-12-01
9013,0x9488c6fa9e9f92d4bad07cb641ede2ffc853e495,2022-01-01 00:00:00.000 UTC,6.111482,1,2022-01-01
9014,0x49e2a5af17fb2b37a0ff460217dc7a9ff4f34f90,2022-06-01 00:00:00.000 UTC,31.373608,1,2022-06-01
9015,0x0aead38a4f99f8dae8944d31eb703b6c7cec4381,2022-07-01 00:00:00.000 UTC,274.461344,2,2022-07-01


In [83]:
df_map_account_nodes=run_query(hopr_node_account_mapping)

Waiting for query to finish...


In [84]:
df_map_account_nodes['datestr_delisted']=[convert_blocktime(df_map_account_nodes.datemonth_delisted_node.iloc[i], delta=8) for i in range(0,len(df_map_account_nodes)) ]
df_map_account_nodes['datestr_listed']=[convert_blocktime(df_map_account_nodes.datemonth_listed_node.iloc[i], delta=8) for i in range(0,len(df_map_account_nodes)) ]

In [85]:
df_map_account_nodes

,account,datemonth_delisted_node,datemonth_listed_node,delisted_node_id,node_id,datestr_delisted,datestr_listed
0,0x01bfbcb6a2924b083969ce6237adbbf3bfa7de13,None,2023-02-01 00:00:00.000 UTC,None,16Uiu2HAkwk6gH2v7aSJiXw9SKUKwyWkvH8HtCfjcCHXHf...,NaT,2023-02-01
1,0x01bfbcb6a2924b083969ce6237adbbf3bfa7de13,None,2023-02-01 00:00:00.000 UTC,None,16Uiu2HAmGZLW41THLNqzohjij7XEiDQvfczgqpqLQi2uz...,NaT,2023-02-01
2,0x01bfbcb6a2924b083969ce6237adbbf3bfa7de13,None,2023-02-01 00:00:00.000 UTC,None,16Uiu2HAm7rTbgcai4JSRD3jdAzFauNivXpu75VEN6G3oB...,NaT,2023-02-01
3,0x01bfbcb6a2924b083969ce6237adbbf3bfa7de13,None,2023-02-01 00:00:00.000 UTC,None,16Uiu2HAmDgdDRQ3boXiP9epTfr8BjZR3fVr5AhG9SwCK9...,NaT,2023-02-01
4,0x01bfbcb6a2924b083969ce6237adbbf3bfa7de13,None,2023-02-01 00:00:00.000 UTC,None,16Uiu2HAmSuw7jmVkViMUZmyAoVqK1qNMN1ApJqaQpCEiu...,NaT,2023-02-01
...,...,...,...,...,...,...,...
812,0xfb832019df452f6c2fe081e79d8508acb8860c49,None,2023-06-01 00:00:00.000 UTC,None,16Uiu2HAmL9wun76mnpkQJpTU43v72GMCvEX1KM5xQokKH...,NaT,2023-06-01
813,0x68bb5ac5c5d3f511a7b8f94f39f1f03a0bf8b60a,None,2023-06-01 00:00:00.000 UTC,None,16Uiu2HAm62VhYjnmzGUbuTWA4o8CBvgHCsvMnzbChTD1m...,NaT,2023-06-01
814,0xc0f5214447016b197e3898c0f57bad499382ee2a,None,2023-06-01 00:00:00.000 UTC,None,16Uiu2HAmGmPSqkr4ShYbWD6YWmM1X3eAS9eqiBTjMdzpx...,NaT,2023-06-01
815,0x1bc3b10b5a33bd650c04aa503b99dbe4fd76a71a,None,2023-06-01 00:00:00.000 UTC,None,16Uiu2HAmQAxYtp42mLXCVaDs6PHND78bkQSDDhqp3w9xv...,NaT,2023-06-01


In [82]:
df_map_account_nodes[df_map_account_nodes.account=='0x1f515ff084f84e89c2a82858eccf421029769e35'].delisted_node_id.nunique()

105

In [80]:
df_map_account_nodes_month[df_map_account_nodes_month.account=='0x1f515ff084f84e89c2a82858eccf421029769e35']

,datestr_delisted,account,num_nodes_listed,num_nodes_delisted,cs_num_nodes_delisted
12,2022-11-01,0x1f515ff084f84e89c2a82858eccf421029769e35,15,15,15
17,2022-12-01,0x1f515ff084f84e89c2a82858eccf421029769e35,73,73,88
22,2023-01-01,0x1f515ff084f84e89c2a82858eccf421029769e35,2,2,90
26,2023-02-01,0x1f515ff084f84e89c2a82858eccf421029769e35,4,4,94
35,2023-03-01,0x1f515ff084f84e89c2a82858eccf421029769e35,9,9,103
38,2023-04-01,0x1f515ff084f84e89c2a82858eccf421029769e35,3,3,106
94,NaT,0x1f515ff084f84e89c2a82858eccf421029769e35,67,0,0


In [105]:
df_map_account_nodes_month_delisted = df_map_account_nodes.groupby(['datestr_delisted','account']).agg(
    num_nodes_listed = ('node_id','nunique'),
    num_nodes_delisted = ('delisted_node_id','nunique')
).reset_index()

In [99]:
df_map_account_nodes_month_listed = df_map_account_nodes.groupby(['datestr_listed','account'],dropna=False).agg(
    num_nodes_listed = ('node_id','nunique')
).reset_index()

In [90]:
df_map_account_nodes_month_delisted

,datestr_delisted,account,num_nodes_listed,num_nodes_delisted
0,2022-09-01,0x56d53a4402a7b8f4ad027cfb421ff60e865f665b,1,1
1,2022-09-01,0x5bffcd8f4809e29bcace4c9c21e0c3b6963ecec1,1,1
2,2022-09-01,0x5e14f9ba0f2aa3d3a11cf2823d0e45f0636a8baf,1,1
3,2022-09-01,0x7d3a7ad44bfcfc0962709ece4c149128717906b8,2,2
4,2022-10-01,0x2402da10a6172ed018aeea22ca60ede1f766655c,1,1
...,...,...,...,...
379,NaT,0xfe5d05fc007b3c1766aa0fddda4a908a1695558d,1,0
380,NaT,0xfea310e868466201e881866e75232e1bc2b86e0c,1,0
381,NaT,0xfed242184ba67298d7c0e7c228da4512abcf5e94,1,0
382,NaT,0xff78bad7f3fefbf552a2978e1455d0126da9ba76,1,0


In [106]:
def make_node_cs_count(df,col):
    df['cs_num_nodes_'+ col] = 0
    for a in df.account.unique():
        tmp=df[(df.account==a)&~(df['datestr_' + col].isna())].sort_values(by=['datestr_' + col])
        if tmp.shape[0]>0:
            df.loc[(df.account==a)&~(df['datestr_' + col].isna()),'cs_num_nodes_'+ col] = tmp['num_nodes_' + col].cumsum()

In [107]:
make_node_cs_count(df_map_account_nodes_month_delisted,'delisted')
make_node_cs_count(df_map_account_nodes_month_listed,'listed')

In [108]:
df_map_account_nodes_month_listed

,datestr_listed,account,num_nodes_listed,cs_num_nodes_listed
0,2022-09-01,0x016d31ae8e538ec295b8d9e65736897d2b6cf245,1,1
1,2022-09-01,0x05c9f9ea62c3c69ddf7221446d7dbf4e04b0c408,1,1
2,2022-09-01,0x09e3c4bf50a15225d4087fe089282cb5506dcce6,1,1
3,2022-09-01,0x105086b1f32ffe1fabb3bc01329c3878a291df6f,1,1
4,2022-09-01,0x1b3921746c0a0376b223e616a2c49a090d09ed59,5,5
...,...,...,...,...
380,2023-06-01,0x68bb5ac5c5d3f511a7b8f94f39f1f03a0bf8b60a,1,1
381,2023-06-01,0xc0f5214447016b197e3898c0f57bad499382ee2a,1,4
382,2023-06-01,0xe002692784bafbafe7cfaf6adb03b43850acc9ef,1,1
383,2023-06-01,0xe963140020f05c01d1b19565597a659e03333a5d,1,1


In [109]:
df_map_account_nodes_month_delisted

,datestr_delisted,account,num_nodes_listed,num_nodes_delisted,cs_num_nodes_delisted
0,2022-09-01,0x56d53a4402a7b8f4ad027cfb421ff60e865f665b,1,1,1
1,2022-09-01,0x5bffcd8f4809e29bcace4c9c21e0c3b6963ecec1,1,1,1
2,2022-09-01,0x5e14f9ba0f2aa3d3a11cf2823d0e45f0636a8baf,1,1,1
3,2022-09-01,0x7d3a7ad44bfcfc0962709ece4c149128717906b8,2,2,2
4,2022-10-01,0x2402da10a6172ed018aeea22ca60ede1f766655c,1,1,1
5,2022-10-01,0x394c82976a57c4309138c072149ee57aea16560b,2,2,2
6,2022-10-01,0x4d8bd62c7cb6769795410c5d1209cc09bfe16c60,1,1,1
7,2022-10-01,0xb5cbb2f3b5d3a72a88af6c3739579a18d5235755,1,1,1
8,2022-10-01,0xeebcfef507d6109d87689811f5d85d89d30cae76,1,1,1
9,2022-11-01,0x053e1be1151630ef9a54e0bcb7f5ccd1503d5725,1,1,1


In [113]:
df_map_account_nodes_month_listed['cs_num_nodes_delisted']=0
for a  in df_map_account_nodes_month_listed.account.unique():
    tmp=df_map_account_nodes_month_listed[(df_map_account_nodes_month_listed.account == a)]
    tmp2=df_map_account_nodes_month_delisted[(df_map_account_nodes_month_delisted.account == a)]
    if tmp2.shape[0]>0:
        for m in tmp.datestr_listed.unique():
            if tmp2[tmp2.datestr_delisted<=m].shape[0]>0:
                df_map_account_nodes_month_listed.loc[(df_map_account_nodes_month_listed.account == a)&(df_map_account_nodes_month_listed.datestr_listed == m),'cs_num_nodes_delisted'] = tmp2[tmp2.datestr_delisted<=m].cs_num_nodes_delisted.iloc[-1]

In [115]:
df_map_account_nodes_month_listed['nodes_live']=df_map_account_nodes_month_listed['cs_num_nodes_listed']-df_map_account_nodes_month_listed['cs_num_nodes_delisted']

In [122]:
df_map_account_nodes_month_listed_grp = df_map_account_nodes_month_listed.groupby('datestr_listed').agg(
    num_nodes_listed = ('nodes_live','sum')
).reset_index()


In [123]:
df_map_account_nodes_month_listed_grp

,datestr_listed,num_nodes_listed
0,2022-09-01,108
1,2022-10-01,69
2,2022-11-01,136
3,2022-12-01,120
4,2023-01-01,81
5,2023-02-01,179
6,2023-03-01,213
7,2023-04-01,222
8,2023-05-01,192
9,2023-06-01,81


In [117]:
# that mapping seems not to work, use df_map_account_nodes_month_listed_grp instead
df_hopr_rewards['nodes_live']=0
for a in df_hopr_rewards.account.unique():
    tmp = df_hopr_rewards[df_hopr_rewards.account==a]
    tmp2=df_map_account_nodes_month_listed[df_map_account_nodes_month_listed.account==a]
    for m in tmp.datestr.unique():
        if tmp2[tmp2.datestr_listed<=m].shape[0]>0:
            df_hopr_rewards.loc[(df_hopr_rewards.account==a)&(df_hopr_rewards.datestr==m),'nodes_live']=tmp2[(tmp2.datestr_listed<=m)].nodes_live.iloc[-1]


In [127]:
df_hopr_rewards_month =df_hopr_rewards.groupby('datestr').agg(
    num_accounts = ('account','nunique'),
    # num_nodes_listed = ('nodes_live','sum'), 
    sum_rewards = ('sum_rewards','sum')
).reset_index()

In [128]:
df_hopr_rewards_month = df_hopr_rewards_month.merge(df_map_account_nodes_month_listed_grp, how='left', left_on='datestr', right_on='datestr_listed')

In [129]:
df_hopr_rewards_month

,datestr,num_accounts,sum_rewards,datestr_listed,num_nodes_listed
0,2021-07-01,116,6.381316e+03,NaT,NaN
1,2021-08-01,315,1.117366e+05,NaT,NaN
2,2021-09-01,348,1.701375e+05,NaT,NaN
3,2021-10-01,363,2.450535e+05,NaT,NaN
4,2021-11-01,394,2.740784e+05,NaT,NaN
5,2021-12-01,355,2.996583e+05,NaT,NaN
6,2022-01-01,755,1.966726e+06,NaT,NaN
7,2022-02-01,471,4.493391e+05,NaT,NaN
8,2022-03-01,382,5.661995e+05,NaT,NaN
9,2022-04-01,639,1.024312e+06,NaT,NaN


In [130]:
df_hopr_rewards_month.to_csv('hopr_rewards_monthly.csv')

In [10]:
# df_hopr_rewards.to_pickle('df_hopr_rewards.pkl')